# Imports and other setup

In [ ]:
import sys
sys.path.append('../')
from py_files.network_creator import *

## Creating, initially inspecting and saving subreddits

Example filepath: '../data/19march/graphs/politics_network.gexf'

Make sure the filepath exists before running (manually create them) as otherwise it will take ~12 mins to run and end up giving an error

### r/politics - takes ~11 mins with 19march politics

In [ ]:
# # grabbing scraped df
# df_politics = pd.read_csv('../data/23march_chur/scrapes/politics.csv')

# G_politics = full_pipeline(df_politics)

# # saving network
# nx.write_gexf(G_politics, '../data/23march_chur/graphs/politics_nan.gexf')

### r/music - takes ~8 mins with 2april music

In [ ]:
# # grabbing scraped df
# df_music = pd.read_csv('../data/2april/scrapes/2apr_2Music.csv')

# G_music = full_pipeline(df_music)

# # saving network
# nx.write_gexf(G_music, '../data/2april/graphs/music.gexf')

### r/antiwork

In [ ]:
# # grabbing scraped df
# df_antiwork = pd.read_csv('../data/17april/scrapes/antiwork.csv')

# G_antiwork = full_pipeline(df_antiwork)

# # saving network
# nx.write_gexf(G_antiwork, '../data/17april/graphs/antiwork.gexf')

### r/gaming - takes ~20 mins with 4april gaming

In [ ]:
# # grabbing scraped df
# df_gaming = pd.read_csv('../data/4april/scrapes/gaming.csv')

# G_gaming = full_pipeline(df_gaming)

# # saving network
# nx.write_gexf(G_gaming, '../data/4april/graphs/gaming.gexf')

# Recreating networks without rounding title embeds

In [ ]:
scrape_list = [
    '../data/date_folders/april_24/scrapes/communism.csv',
    '../data/date_folders/april_23/scrapes/PoliticalDiscussion.csv',
    '../data/date_folders/april_23/scrapes/Republican.csv',
    '../data/date_folders/april_23/scrapes/democrats.csv',
    '../data/date_folders/april_23/scrapes/ukpolitics.csv',
    '../data/date_folders/april_23/scrapes/worldnews.csv',
    '../data/date_folders/april_18/scrapes/CallOfDuty.csv',
    '../data/date_folders/april_18/scrapes/FIFA.csv',
    '../data/date_folders/april_17/scrapes/antiwork.csv',
    '../data/date_folders/april_4/scrapes/gaming.csv',
    '../data/date_folders/march_23/scrapes/politics.csv',
    '../data/date_folders/march_19/scrapes/EscapefromTarkov.csv',
    '../data/date_folders/may_7/scrapes/Cooking.csv'
]

save_paths = [
    '../data/date_folders/april_24/graphs/communism.gexf',
    '../data/date_folders/april_23/graphs/PoliticalDiscussion.gexf',
    '../data/date_folders/april_23/graphs/Republican.gexf',
    '../data/date_folders/april_23/graphs/democrats.gexf',
    '../data/date_folders/april_23/graphs/ukpolitics.gexf',
    '../data/date_folders/april_23/graphs/worldnews.gexf',
    '../data/date_folders/april_18/graphs/CallOfDuty.gexf',
    '../data/date_folders/april_18/graphs/FIFA.gexf',
    '../data/date_folders/april_17/graphs/antiwork.gexf',
    '../data/date_folders/april_4/graphs/gaming.gexf',
    '../data/date_folders/march_23/graphs/politics.gexf',
    '../data/date_folders/march_19/graphs/tarkov.gexf',
    '../data/date_folders/may_7/graphs/Cooking.gexf'
]

In [ ]:
for scrape_path, save_path in zip(scrape_list, save_paths):
    print(scrape_path)
    print(save_path)
    df = pd.read_csv(scrape_path)
    G = full_pipeline(df)
    nx.write_gexf(G, save_path)

In [ ]:
# grabbing scraped df
df_cooking = pd.read_csv('../data/date_folders/may_7/scrapes/Cooking.csv')
G_cooking = full_pipeline(df_cooking)
nx.write_gexf(G_cooking, '../data/date_folders/may_7/graphs/Cooking.gexf')

In [ ]:
# grabbing scraped df
df_antiwork = pd.read_csv('../data/date_folders/april_24/scrapes/communism.csv')

G_antiwork = full_pipeline(df_antiwork)

# saving network
nx.write_gexf(G_antiwork, '../data/date_folders/april_24/graphs/communism.gexf')

In [ ]:
# grabbing scraped df
df_antiwork = pd.read_csv('../data/date_folders/april_18/scrapes/CallOfDuty.csv')

G_antiwork = full_pipeline(df_antiwork)

# saving network
nx.write_gexf(G_antiwork, '../data/date_folders/april_18/graphs/CallOfDuty.gexf')

In [ ]:
# grabbing scraped df
df_antiwork = pd.read_csv('../data/date_folders/april_18/scrapes/FIFA.csv')

G_antiwork = full_pipeline(df_antiwork)

# saving network
nx.write_gexf(G_antiwork, '../data/date_folders/april_18/graphs/FIFA.gexf')